#                            DATA EXPLORATION & CLEAN-UP

## Acquiring Airport Code of Departure & Destination Cities

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv
from tqdm import tqdm_notebook as tqdm

In [ ]:


#-----------------------------####--------------------------------------------

#Getting CSV file of the most populated cities in the world
csv_path = "Resources/Airport Codes/most_populated_cities_csv.csv"
most_populated_cities_df = pd.read_csv(csv_path,encoding="utf-8")
most_populated_cities_df.head()

#Getting CSV file of the most visited cities in the world
csv_path2 = "Resources/Airport Codes/100_Most_visited_cities.csv"
most_visited_cities_df = pd.read_csv(csv_path2,encoding="utf-8")
most_visited_cities_df.head()


###-----------------------------#####--------------------------------

## Data Acquisitio Outbound Cities#####

#List of outbound cities
outbound_cities = ['Houston','Chicago','New York','Atlanta','Los Angeles','Seattle']

#Empty list to append data acquired
outbound_name = []
destination_country = []
airportID =[]
airportName = []

#City counter kick off. 
city_counter = 1

#Base url
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/USA/USD/en-US/?query="

#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition of Departure City (Airport Code)")
print(f"------------&&&&-------------------")

for i in tqdm(outbound_cities):
    try:
        #Getting the API with the first outbound city in the list (Houston) and loop thru the list
        response = requests.get(url+i,
          headers={
            "X-RapidAPI-Host": "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
            "X-RapidAPI-Key": "09ec0160admsh2d5563a7feaa3efp1e2db5jsn5787845ed7fc"
          }
    ).json()
        #Export the name and the ID
        outbound_name.append(response["Places"][0]["PlaceName"])
        airportID.append(response["Places"][1]["PlaceId"])
        airportName.append(response["Places"][1]["PlaceName"])
        current_city = response["Places"][0]["PlaceName"]
        
        print(f"Data Acquisition {city_counter} | Outbound City: {current_city}")  
        city_counter = city_counter + 1
    except:
        print("Not a major outbound city")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")

#---------------------############------------------------------#
#Creating a data frame of the data acquired
outbound_data = {"City Name": outbound_name, "Airport Name": airportName, "Airport Code": airportID}
outbound_data_df = pd.DataFrame(outbound_data)
#Exporting data frame to csv file
export_csv = outbound_data_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\City_departure_dataFinal.csv',index = False)
#Transform the Airport code data into a list in order to used for the other API
outbound_airport_code_list = outbound_data_df['Airport Code'].tolist()

#---------------------###########--------------------------------#

#Reading the Most visited Cities file#

#Converting City Name into a list to gather airport code
#https://stackoverflow.com/questions/22341271/get-list-from-pandas-dataframe-column/22341390
city_destination_most_visited_list = most_visited_cities_df["City Name"].tolist()

city_destination_most_visited_list

#--------------------------------#########-------------------------------------------

## Destination Most Visited Cities ######

#Empty list to store data
visited_destination_name = []
visited_country = []
visited_destination_airportID =[]
visited_destination_airportName = []

visited_dest_city_counter = 1

#Base URL
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/USA/USD/en-US/?query="
#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition of Destination City and respective Airport Codes")
print(f"------------&&&&-------------------")

for z in tqdm(city_destination_most_visited_list):
    try:
        response = requests.get(url+z,
          headers={
            "X-RapidAPI-Host": "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
            "X-RapidAPI-Key": "09ec0160admsh2d5563a7feaa3efp1e2db5jsn5787845ed7fc"
          }
    ).json()
        visited_destination_name.append(response["Places"][0]["PlaceName"])
        visited_country.append(response["Places"][0]["CountryName"])
        visited_destination_airportID.append(response["Places"][1]["PlaceId"])
        visited_destination_airportName.append(response["Places"][1]["PlaceName"])
        visited_destination_city = response["Places"][0]["PlaceName"]
        
        print(f"Data Acquisition {visited_dest_city_counter} | Outbound City: {visited_destination_city}")  
        visited_dest_city_counter = visited_dest_city_counter + 1
    except:
        print("Not a good city to travel... try another city")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")

##-----------------------------####------------------------------------------------------------------------

#Creating data frame of Airport Name and Airport Code of the data acquired
most_visited_destination_data =  {"Airport Name": visited_destination_airportName, "Airport Code": visited_destination_airportID }
most_visited_destination_data = pd.DataFrame(most_visited_destination_data)

#Exporting file to CSV format
export_csv = most_visited_destination_data.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\most_visited_destination_dataFINAL.csv',index = False)

#most_visited_destination_data.head()

## Acquiring travel routes

In [ ]:
#Reading outbound cities data csv 
outbound_data = "Resources/Airport Codes/City_departure_dataFinal.csv"
outbound_data_df = pd.read_csv(outbound_data,encoding="utf-8")
outbound_data_df.head()

#Reading csv of most visited cities
most_visited_destination = "Resources/Airport Codes/most_visited_destination_dataFINAL.csv"
most_visited_cities_df = pd.read_csv(most_visited_destination,encoding="utf-8")

#Creating a list of the name for the outbound airport cities
outbound_airport_code_tolist = outbound_data_df['Airport Code'].tolist()

#Creating a list for name of the airport of the most visited cities
most_visited_cities_airport_list = most_visited_cities_df['Airport Code'].tolist()

#------------------------------------###--------------------------------------------------------

# Routes for all most visitied cities and one city outbound and fixed dates

#Empty list to store data
carriers_info = []
places_info = []
quotes_info = []

#Base API url for routes
base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browseroutes/v1.0/US/USD/en-US/"

headers={"X-RapidAPI-Host": "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    "X-RapidAPI-Key": "077cfd7813mshfe97bf766bd036dp106fd9jsneb6caf492da9"}

#Variables for the url
originplace = outbound_airport_code_tolist
destinationplace = most_visited_cities_airport_list
outboundpartialdate = ["2019-12-15"]
inboundpartialdate = ["2020-01-05"]

#Start counter for the loop
destination_counter = 1

#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition from {outbound_airport_code_tolist[1]} airport to the most visited cities in the world !!!")
print(f"------------&&&&-------------------")

for x in tqdm(destinationplace):
    try:
        response = requests.get(base_url+outbound_airport_code_tolist[1]+"/"+x+ "/"+outboundpartialdate[0]+"?inboundpartialdate="+inboundpartialdate[0], headers=headers).json()
        #Append values to the empty list
        carriers_info.append(response['Carriers'])
        places_info.append(response['Places'])
        quotes_info.append(response['Quotes'])
        
        print(f"Data Acquisition {destination_counter} | Outbound City: {outbound_airport_code_tolist[1]}")  
        
        destination_counter = destination_counter + 1
    except:
        print("Route not found...!!")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")      

################################################################################################################

#Loop thru the carriers information data that contains independent dictionaries and append results to a empty list
carriers_list = []
for x in range(len(carriers_info)):
    for y in range(len(carriers_info[x])):
        #print(carriers_info[x][y])
        carriers_list.append(carriers_info[x][y])
        

#Loop thru the places information data that contains dictionaries and append results to a empty list
places_list = []
for i in range(len(places_info)):
    for j in range(len(places_info[i])):
        #print(places_info[i][j])
        places_list.append(places_info[i][j])
        
#Loop thru the quoute information data that contains nested dictionaries and append results to a empty list
#Noted the nested dictionary of the outboundleg which is append to the outbound list
quotes_list = []
outbound_list = []
for z in range(len(quotes_info)):
    for w in range(len(quotes_info[z])):
        quotes_list.append(quotes_info[z][w])
        outbound_list.append(quotes_info[z][w]['OutboundLeg'])    
        
##################################################################################################################3

#Creating the data frame of carrriers
carriers_df = pd.DataFrame(carriers_list)
carriers_df
#Exporting dataframe into csv format
export_csv = carriers_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\carriers_df.csv',index = False)
len(carriers_df)

#######################################################################################################################
#Creating the data frame for places
places_df = pd.DataFrame(places_list)
places_df
#Exporting dataframe into csv format
export_csv = places_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\placescsv.csv',index = False)
places_df

####################################################################################################################
#Create the data frame of the quotes list
quotes_df = pd.DataFrame(quotes_list)
quotes_df.head()

#Create the data frame of the outbound list
outbound_df = pd.DataFrame(outbound_list)
outbound_df

#Merged both data frames together on the index since data is symetric
merged_quotes_outbound = pd.merge(quotes_df,outbound_df, left_index = True, right_index = True)
merged_quotes_outbound

#Export to CSV 
export_csv = merged_quotes_outbound.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\merged_quotes_outbound.csv',index = False)
#merged_quotes_outbound

# Data Cleaning

## Reading & Cleaning Data

In [ ]:
# ################ Reading CSV ##################################

# #Reading carriers data csv 
# carriers_data = "Resources/Seattle Data/SE_carriers_df.csv"
# carriers_data_df = pd.read_csv(carriers_data,encoding="utf-8")

# # Merge Airline Name, keep in mind that a dictionary has been created for the data frame of carriers to eliminate duplicates
# #https://stackoverflow.com/questions/20250771/remap-values-in-pandas-column-with-a-dict
# #This is the dictionary
# airlines_dict = {row[0]: row[1] for k, row in carriers_data_df.iterrows()}
# airlines_dict




In [ ]:

# ########################### Carriers Data ##################################################



# #Renaming columns
# #https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas
# carriers_data_df = carriers_data_df.rename(columns={'CarrierId': 'Airline ID', 'Name':'Airline Name'})

# convert_dict2 = {'Airline ID': object}
# carriers_data_df = carriers_data_df.astype(convert_dict2) 

In [ ]:
carriers_data_df.head()

In [ ]:

# ########################## Quotes Data #######################################################


# #Reading Quoutes data csv
# quotes_data = "Resources/Seattle Data/SE_merged_quotes_outbound.csv"
# quotes_data_df = pd.read_csv(quotes_data,encoding="utf-8")
# quotes_data_df.head()

# #Delete columns that we don't use
# #https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/
# quotes_data_df = quotes_data_df.drop(['OutboundLeg','QuoteDateTime'], axis =1)

# #Delete the brackets from carrier Ids
# quotes_data_df['CarrierIds'] = quotes_data_df['CarrierIds'].str.replace("[","")
# quotes_data_df['CarrierIds'] = quotes_data_df['CarrierIds'].str.replace("]","")

# #Delete the time from the daparture date
# quotes_data_df['DepartureDate'] = quotes_data_df['DepartureDate'].str.replace("T00:00:00","")

# #Change order of columns 
# #https://erikrood.com/Python_References/change_order_dataframe_columns_final.html
# quotes_data_df = quotes_data_df[['OriginId','DestinationId','MinPrice','CarrierIds','Direct','DepartureDate','QuoteId']]

# #Renaming columns
# #https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas
# quotes_data_df = quotes_data_df.rename(columns={'OriginId': 'City ID Departure', 'DestinationId':'City ID','MinPrice':'Min Price', 'CarrierIds': 'Airline ID', 'Direct': 'Non-Stop Flight', 'DepartureDate':'Departure Date', 'QuoteId': 'Quote ID'})

# #Change Airline ID to numeric
# #https://www.geeksforgeeks.org/change-data-type-for-one-or-more-columns-in-pandas-dataframe/
# # convert_dict = {'Airline ID': object}
# # quotes_data_df = quotes_data_df.astype(convert_dict) 
# quotes_data_df.head()


In [ ]:
places_df.shape

In [ ]:
# ################################### Places Data ######################################################################

# #Reading places data csv
# places_data = "Resources/Seattle Data/SEplacescsv.csv"
# places_data_df = pd.read_csv(places_data,encoding="utf-8")
# places_data_df.head()

# #Drop/remove entire column CityId, Type, drop SkyscannerCode, change PlaceId to string
# #Delete columns that we don't use
# #https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/
# places_data_df = places_data_df.drop(['CityId','Type','SkyscannerCode'], axis =1)

# #Change order of columns 
# #https://erikrood.com/Python_References/change_order_dataframe_columns_final.html
# places_data_df = places_data_df[['PlaceId','CityName','Name','IataCode','CountryName']]

# #Renaming columns
# #https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas
# places_data_df = places_data_df.rename(columns={'CityName': 'City Name', 'CountryName':'Country Name','IataCode':'Iata Code', 'Name': 'Airport Name', 'PlaceId': 'City ID'})




In [ ]:
# #################################### Merging ###########################################################################

# #Merge quoutes and places
# merge_1 = pd.merge(places_data_df, quotes_data_df, how = 'outer', on = 'City ID' )
# #merge_1.head(1000)

# #Drop NA values
# #http://www.datasciencemadesimple.com/drop-rows-with-nan-na-drop-missing-value-in-pandas-python-2/
# merge_1 = merge_1.dropna()



In [ ]:
# # Map the city name to the Airline ID by creating a new Airline Name column
# merge_1['Airline Name'] = merge_1['Airline ID'].map(airlines_dict)
# merge_1


In [ ]:
# ############################################ Export to CSV ###############################################################

# #Export and save to CSV
# export_csv = merge_1.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\MergeCleanFINAL.csv',index = False)


# #Export clean data frame to csv
# export_csv = places_data_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\placesCLEAN.csv',index = False)

# #Export clean data frame to csv
# export_csv = quotes_data_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\quotesCLEAN.csv',index = False)


# #Export clean data frame to csv
# export_csv = carriers_data_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\carriersCLEAN.csv',index = False)